<div style="font-size: smaller; color: #808080; text-align: center; display: block;">MIT - 6.802 / 6.874 / 20.390 / 20.490 / HST.506 Computational Systems Biology: Deep Learning in the Life Sciences - Spring 2019</div>

# Problem Set 2: Transcription factor binding site prediction with CNNs (and RNNs)

In this problem set we are going to train a CNN to classify the binding sequences of the transcription factor CTCF. The positive examples are 101 bp DNA sequences centered at CTCF ChIP-seq peaks from GM12878 cells. Negative sequences are generated by permuting the nucleotides in the positive sequences while keeping the di-nucleotide frequency. All the sequences have been embedded using one-hot encoding. Unlike problem set 1, which was a multi-class classification task (10 digits), problem set 2 is a binary classification task (CTCF is binding, CTCF is not binding).

Concepts and techniques relevant to this problem set:
- convolutional layers
- recurrent layers (for 6.874 students only)
- model evaluation with ROC and precision-recall curves
- model serialization and deserialization (storing models on disk)
- grid search as hyperparameter optimization technique

In [1]:
# import libraries, set random seed for reproducibility, and print TensorFlow version

# if libraries are missing:
# !pip install tensorflow
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !pip install h5py
# !pip install sklearn
# !pip install tqdm

import random
import sys
import os
import shutil

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import h5py
import sklearn.metrics
from tqdm import tqdm_notebook as tqdm

random.seed(0)
print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# function to load the data embedded in the previous problem and their labels 
def load_data(prefix='', path='./data/'):
    training_set = h5py.File(os.path.join(path, prefix + 'train.h5'), 'r')
    validation_set = h5py.File(os.path.join(path, prefix + 'valid.h5'), 'r')
    test_set = h5py.File(os.path.join(path, prefix + 'test.h5'), 'r')
    
    return(training_set['data'][:], training_set['label'][:],
           validation_set['data'][:], validation_set['label'][:],
           test_set['data'][:], test_set['label'][:])

In [ ]:
# Load the data
SEQ_LEN = 101
NUM_BASES = 4
NUM_CLASSES = 2

x_tr, y_tr, x_va, y_va, x_te, y_te = load_data()
print("training set:", x_tr.shape)
print("validation set:", x_va.shape)
print("test set:", x_te.shape)

## Problem 1: Construct a CNN with given specifications (50 points)

Implement a CNN model with the following architecture: 
1. one convolutional layer with 32 kernels (filters) of width 11 and height 1, step size of 1 and ReLU activation. The input should be padded so that the output has the same size as the original input.
2. one max-pooling layer with pooling size of 2
3. flatten the tensor to prepare it for the fully connected layer
4. one fully connected layer with 64 neurons and ReLU activation
5. one dropout layer with a `dropout_rate` (dropout rate = 1 - keep probability)
6. one fully connected layer with 2 neurons and sigmoid activation
7. all the weights (not the biases) in the network should be returned in list `weights` (for L2 regularization in function `cnn_loss`)

About weight initialization:
- do not initialize weights with zeros or other constant values!
- use random initialization to [break symmetries](https://medium.com/usf-msds/deep-learning-best-practices-1-weight-initialization-14e5c0295b94), e.g.
  - zero-centered Gaussian with low variance (e.g., $\sigma = 0.01$)
  - zero-centered truncated Gaussian with low variance (e.g., $\sigma = 0.01$)
  - Xavier intialization
  - He initialization

About bias initialization:
- can be initialized to constant value, also zero (since weight initialization breaks symmetry)
- commonly initialized to 0.1, 0.01, 0.0, 1.0

Useful functions:
- [tf.truncated_normal](https://www.tensorflow.org/api_docs/python/tf/truncated_normal)
- [tf.nn.conv2d](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d)
- [tf.nn.bias_add](https://www.tensorflow.org/api_docs/python/tf/nn/bias_add)
- [tf.nn.relu](https://www.tensorflow.org/api_docs/python/tf/nn/relu)
- [tf.nn.max_pool](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool)
- [tf.nn.dropout](https://www.tensorflow.org/api_docs/python/tf/nn/dropout)
- [tf.nn.softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax)
- [tf.matmul](https://www.tensorflow.org/api_docs/python/tf/matmul)

In [ ]:
def cnn_model(x_ph, dropout_rate):
    ################################################################################ 
    # Implement a CNN with the required structure and initialize all the variables:
    # Arguments:
    # - x_ph: one-hot encoded DNA input data / placeholder of shape [?, 1, 101, 4]
    # - dropout_rate: dropout rate (1 - keep probability) of dropout layer
    # Return values:
    # - y_hat_op: class probabilities operator (our predicted y, sigmoid(z_op)) of shape [?, 2]
    # - z_op: unscaled log probabilities operator (output of last matrix multiplication,
    #      before activation function) of shape [?, 2]
    # - weights: a list of all tf.Variable weight matrices
    ################################################################################
    # x_ph: [batch, in_height, in_width, in_channels] -> [?, 1, 101, 4]
    # conv layer filter: [filter_height, filter_width, in_channels, out_channels] -> [1, 11, 4, 32]
    conv_filter_height = 1
    conv_filter_width = 11
    in_channels = 4
    out_channels = 32
    fc_units = 64
    max_pool_stride = 2
    ################################################################################
    #                            BEGINNING OF YOUR CODE                            #
    ################################################################################
    
    
    
    ################################################################################
    #                               END OF YOUR CODE                               #
    ################################################################################
    return y_hat_op, z_op, weights

Useful functions:
- [tf.nn.softmax_cross_entropy_with_logits_v2](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits_v2)
- [tf.nn.l2_loss](https://www.tensorflow.org/api_docs/python/tf/nn/l2_loss)
- [tf.reduce_mean](https://www.tensorflow.org/api_docs/python/tf/reduce_mean)

In [ ]:
def cnn_loss(z_op, y_ph, weights, l2_coefficient):
    ################################################################################ 
    # Implement a loss operator, which calculates an L2 regularized cross entropy 
    # loss 
    # Arguments:
    # - z_op: unscaled log probabilities (output of last layer) of shape [?, 2]
    # - y_ph: labels / placeholder of shape [?, 2]
    # - weights: list of all weight tf.Variables
    # - l2_coefficient: coefficient lambda for L2 regularization of weights
    # Return values:
    # - loss_op: the loss operator
    ################################################################################
    #                            BEGINNING OF YOUR CODE                            #
    ################################################################################
    
    
    
    ################################################################################
    #                               END OF YOUR CODE                               #
    ################################################################################
    return loss_op

## Problem 2: Implement the training loop (30 points)

In order to perform hyper-parameter search later, we need a training function that takes the hyperparameter configuration as input parameter.

Implement `training`:
- initialize all variables
- train for `num_epochs` epochs. In each epoch, train on mini-batch of size `batch_size`.
- at the end of each epoch:
  - calculate validation loss
  - save model if validation loss is lower than validation loss of previous epochs (inputs: `x_ph`, `y_ph`; output: `y_hat_op`, `loss_op`)

Useful functions:
- [tf.saved_model.simple_save](https://www.tensorflow.org/api_docs/python/tf/saved_model/simple_save)

[How to save a model with simple_save](https://stackoverflow.com/questions/33759623/tensorflow-how-to-save-restore-a-model/50852627#50852627). Don't forget to name operators that you what to restore later (e.g., `y_hat_op`).

In [ ]:
def training(x_train, y_train, x_val, y_val, hyperparam_config, num_epochs, batch_size,
             save_model=True, model_dir='models/best_model'): 
    ################################################################################
    # Arguments:
    # - x_train: input training set
    # - y_train: label training set
    # - x_val: input validation set
    # - y_val: label validation set
    # - hyperparam_config: a dictionary that stores a hyperparameter configuration,
    #                      including:
    #                      - "dropout_rate": dropout rate (1 - keep probability),
    #                      - "l2": coefficient lambda for L2 regularization,
    #                      - "lr": learning rate for RMSProp optimizer
    # - num_epochs: number of epochs to train
    # - batch_size: training mini-batch size
    # - best_model_dir: location where model will be saved
    # Return values:
    # - best_loss: best validation loss
    ################################################################################
    tf.reset_default_graph()
    
    # define input and output placeholders
    # TF uses NHWC: the data is stored in the order of [batch, in_height, in_width, in_channels]
    # DNA is one-dimensional (height = 1, width = 101) with four channels for the four bases A, C, G, T
    x_ph = tf.placeholder("float", [None, 1, SEQ_LEN, NUM_BASES], name="x_ph")
    y_ph = tf.placeholder("float", [None, NUM_CLASSES], name="y_ph")

    # model
    y_hat_op, z_op, weights = cnn_model(x_ph, hyperparam_config['dropout_rate'])
    
    # loss
    loss_op = cnn_loss(z_op, y_ph, weights, hyperparam_config['l2'])

    # optimizer
    optimizer_op = tf.train.RMSPropOptimizer(hyperparam_config['lr'], 0.9).minimize(loss_op)
     
    # Implement training loop
    # - loop through training data num_epochs times
    # - after each epoch, calculate loss on validation set (x_val, y_val)
    # - if save_model: save model with lowest validation loss to disk (model_dir)
    # - (delete previous best model)
    # - return lowest validation loss
    ################################################################################
    #                            BEGINNING OF YOUR CODE                            #
    ################################################################################
    
    
    
    ################################################################################
    #                               END OF YOUR CODE                               #
    ################################################################################
    return best_loss

## Problem 3: Hyperparameter search (10 points)

In the following we will call the `training` function multiple times with different hyperparameter configurations. We first define a handful of possible values for each hyperparameter and then test all combinations. This most basic version of hyperparameter optimization is called grid search.

To save time, we will only train on a subsample of the training set (`num_training`) and evaluate the loss on a subsample of the validation set (`num_validation`).

In [ ]:
def subsample_set(x, y, n):
    ################################################################################ 
    # Draws n examples without replacement from sets x, y
    # Arguments:
    # - x: input set
    # - y: label set
    # - n: number of examples drawn
    # Return values:
    # - x_subset: randomly selected x
    # - y_subset: corresponding labels of randomly selected x
    ################################################################################
    idx = np.arange(len(x))
    np.random.shuffle(idx)
    idx = idx[0 : n]
    return x[idx], y[idx]

In [ ]:
def grid_search(x_train, y_train, x_val, y_val, dropout_rates, l2_lambdas, learning_rates,
                num_training, num_validation, num_epochs=5, batch_size=128):
    ################################################################################ 
    # Arguments:
    # - x_train: input training set
    # - y_train: label training set
    # - x_val: input validation set
    # - y_val: label validation set
    # - dropout_rates: dropout rates to try
    # - l2_lambdas: L2 lambda coefficients to try
    # - learning_rates: learning rates to try
    # - num_training: number of training examples used per model
    # - num_validation: number of validation examples used per model
    # - num_epochs: number of epochs to train
    # - batch_size: training mini-batch size
    # Return values:
    # - losses: losses for configurations tested
    ################################################################################
    losses = []
    
    pbar = tqdm(total = len(dropout_rates) * len(l2_lambdas) * len(learning_rates))
    for dropout_rate in dropout_rates:
        for l2_lambda in l2_lambdas:
            for learning_rate in learning_rates:
                # subsample training and validation sets
                subset_x_train, subset_y_train = subsample_set(x_train, y_train, num_training)
                subset_x_val, subset_y_val = subsample_set(x_val, y_val, num_validation)
                
                hyperparam_config = {'dropout_rate': dropout_rate,
                                     'l2': l2_lambda,
                                     'lr': learning_rate}
                
                best_loss = training(subset_x_train, subset_y_train,
                                     subset_x_val, subset_y_val,
                                     hyperparam_config,
                                     num_epochs,
                                     batch_size,
                                     save_model = False)
                
                losses.append([dropout_rate, l2_lambda, learning_rate, best_loss])
                pbar.update(1)
    pbar.close()
    return losses

In [ ]:
# hyperparameter values to test
dropout_rates = [1.0 - 1e-3, 0.8, 0.5, 0.2]
l2_lambdas = [1e-03, 1e-06, 0]
learning_rates = [1e-1, 1e-3, 1e-5]

# static hyperparameters
num_training = 10000
num_validation = 2000
batch_size = 128
num_epochs = 10

In [ ]:
param_result = grid_search(x_tr, y_tr, x_va, y_va, dropout_rates, l2_lambdas, learning_rates,
                           num_training, num_validation, num_epochs, batch_size)
df = pd.DataFrame(param_result,
                  columns = ['dropout rate', 
                             'L2 lambda',
                             'learning rate',
                             'validation loss']).pivot_table(values = ['validation loss'],
                                                             columns = ['L2 lambda'],
                                                             index = ['dropout rate', 'learning rate'])
df = df.round(3)
(df.style.background_gradient(cmap=sb.cubehelix_palette(start=.5, rot=-.75, as_cmap=True)))

### Questions

#### Question 3.1

Based on your observation, is the performance impacted by only one hyper-parameter or several? Describe your observation to support your conclusion.

<span style="text-transform: uppercase; color: #408080; font-size: bigger; font-weight: bold;">Your (short) answer:</span>

#### Question 3.2

What have you observed when the L2 coefficient is 1e-03? What do you think will happen if we have an even larger coefficient, and why?

<span style="text-transform: uppercase; color: #408080; font-size: bigger; font-weight: bold;">Your (short) answer:</span>

#### Question 3.3

- Assuming each hyper-parameter has a constant number of choices to pick from, how fast does the search space grow as a function of the number of hyper-parameters to consider?
- Can you think of smarter ways than grid search to explore the hyperparameter space?

<span style="text-transform: uppercase; color: #408080; font-size: bigger; font-weight: bold;">Your (short) answer:</span>

## Problem 4: Evaluate best model on test set (10 points)

Now let's train the network with the best hyper-parameters on the **whole** training set:

In [ ]:
best_model_dir = "models/best_model"
################################################################################
#                            BEGINNING OF YOUR CODE                            #
################################################################################
# change dropout rate, L2 coefficient, and learning rate to
# best values (according to grid search above):
hyperparam_config = {'dropout_rate': 0,
                     'l2': 0,
                     'lr': 0}
################################################################################
#                               END OF YOUR CODE                               #
################################################################################
validation_loss = training(x_tr, y_tr, x_va, y_va,
                           hyperparam_config, num_epochs, batch_size, save_model=True, model_dir=best_model_dir)
print('validation loss:', validation_loss)

Implement the `predict` function. Restoring a previously saved model is described [here](https://www.tensorflow.org/guide/saved_model) and [here](https://stackoverflow.com/questions/33759623/tensorflow-how-to-save-restore-a-model/50852627#50852627).

Useful functions:
- [tf.saved_model.load](https://www.tensorflow.org/api_docs/python/tf/saved_model/load)

In [ ]:
def predict(x, model_dir):
    ################################################################################
    # load a trained model and predict y_hat for examples in x
    # Arguments
    # - x: one-hot encoded DNA input data of shape [?, 1, 101, 4]
    # - model_dir: where the model was saved
    # Return values: 
    # - y_hat: class probabilities of examples in x
    ################################################################################
    #                            BEGINNING OF YOUR CODE                            #
    ################################################################################
    
    
    
    ################################################################################
    #                               END OF YOUR CODE                               #
    ################################################################################
    return y_hat

### Model evalutation

We will evaluate the trained model using the `predict` function defined above.

In [ ]:
y_hat_test = predict(x_te, best_model_dir)

#### ROC curve

The area under the ROC curve (auROC) evaluated on the test set should be around 0.94.

In [ ]:
def plot_roc_curve(y, y_hat):
    fpr, tpr, _ = sklearn.metrics.roc_curve(y[:, 0], y_hat[:, 0])
    roc_auc = sklearn.metrics.auc(fpr, tpr)

    plt.figure(figsize=(10,10))
    plt.rcParams.update({'font.size': 14})
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
plot_roc_curve(y_te, y_hat_test)

#### Precision-Recall curve

In [ ]:
def plot_pr_curve(y, y_hat):
    precision, recall, _ = sklearn.metrics.precision_recall_curve(y[:, 0], y_hat[:, 0])
    pr_auc = sklearn.metrics.auc(recall, precision)
    f1 = sklearn.metrics.f1_score(y[:, 0], np.round(y_hat[:, 0]))
    
    plt.figure(figsize=(10,10))
    plt.rcParams.update({'font.size': 14})
    plt.plot(recall, precision, color='darkorange',
             lw=2, label='PR curve (area = {0}, $F_1 = {1}$'.format(
              round(pr_auc, 2), round(f1, 2)))

    f_scores = np.linspace(0.2, 0.8, num=4)
    lines = []
    labels = []
    for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
        plt.annotate('$F_1 = {0:0.1f}$'.format(f_score), xy=(0.87, y[45] + 0.02))

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.0])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve')
    plt.legend(loc="lower left")
    plt.show()

The area under the PR curve should be around 0.95 and $F_1 	\approx 0.87$.

In [ ]:
plot_pr_curve(y_te, y_hat_test)

## Problem 5: CNN + RNN (6.874 only - 20 points out of 120 points)

(Students enrolled in 6.802, 20.390, 20.490 or HST.506 can also submit a solution for this problem - for an extra 20 points)

The binding of some transcription factors depends on the distance-specific existence of co-binding factors. Recall from class that RNNs are good at capturing dependencies between different positions in the input. Thus we are going to explore whether a CNN-RNN hybrid model could achieve a better performance than a regular CNN in classifying motif pairs (NFKB and CTCF in this problem) with variable spacing (10 - 20bp) from the same motif pairs with random spacing in a 101bp sequence. 

**Note**: 
- You can use the best CNN hyperparameters obtained from the previous sections for the CNN part of your model.
- You should be able to achieve an auROC around 0.85 for the CNN and 0.87 for the CNN + RNN model.

In [ ]:
x_tr_rnn, y_tr_rnn, x_va_rnn, y_va_rnn, x_te_rnn, y_te_rnn = load_data(prefix='rnn_', path='./data/')
print(x_tr_rnn.shape, x_va_rnn.shape, x_te_rnn.shape)

### Define model and loss

Adapt the `cnn_model` from before by adding a one-layer LSTM RNN (with 48 units) after the max-pooling layer (instead of the fully-connected layer). We will use the softmax of the output of the last LSTM cell as the probability of the two classes. We will ignore regularization and hyperparameter search for this problem (unless you want to explore that).

Useful functions:
- [tf.nn.conv2d](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d)
- [tf.nn.bias_add](https://www.tensorflow.org/api_docs/python/tf/nn/bias_add)
- [tf.nn.relu](https://www.tensorflow.org/api_docs/python/tf/nn/relu)
- [tf.nn.max_pool](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool)
- [tf.squeeze](https://www.tensorflow.org/api_docs/python/tf/squeeze)
- [tf.transpose](https://www.tensorflow.org/api_docs/python/tf/transpose)
- [tf.split](https://www.tensorflow.org/api_docs/python/tf/split)
- [tf.nn.rnn_cell.LSTMCell](https://www.tensorflow.org/api_docs/python/tf/nn/rnn_cell/LSTMCell)
- [tf.nn.static_rnn](https://www.tensorflow.org/api_docs/python/tf/nn/static_rnn)
- [tf.matmul](https://www.tensorflow.org/api_docs/python/tf/linalg/matmul)
- [tf.nn.softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax)

In [ ]:
def cnn_rnn_model(x_ph):
    ################################################################################ 
    # Implement a CNN-RNN hybrid
    # Arguments:
    # - x_ph: one-hot encoded DNA input data / placeholder of shape [?, 1, 101, 4]
    # Return values:
    # - y_hat_op: class probabilities operator (our predicted y, sigmoid(z_op)) of shape [?, 2]
    # - z_op: unscaled log probabilities operator (output of last matrix multiplication,
    #      before activation function) of shape [?, 2]
    ################################################################################
    # CNN hyperparameters
    conv_filter_height = 1
    conv_filter_width = 11
    in_channels = 4
    out_channels = 32
    
    max_pool_stride = 2
    
    # RNN hyperparameters
    rnn_units = 48
    unroll_length = 51 # complete sequence length (101 after pool stride of 2)
    ################################################################################
    #                            BEGINNING OF YOUR CODE                            #
    ################################################################################
    
    
    
    ################################################################################
    #                               END OF YOUR CODE                               #
    ################################################################################
    return y_hat_op, z_op

`cnn_rnn_loss` is `cnn_loss` without the $L_2$ regularization term.

In [ ]:
def cnn_rnn_loss(z_op, y_ph):
    ################################################################################ 
    # Implement a loss operator, which calculates sigmoid cross entropy loss 
    # Arguments:
    # - z_op: unscaled log probabilities (output of last layer) of shape [?, 2]
    # - y_ph: labels / placeholder of shape [?, 2]
    # Return values:
    # - loss_op: the loss operator
    ################################################################################
    #                            BEGINNING OF YOUR CODE                            #
    ################################################################################
    
    
    
    ################################################################################
    #                               END OF YOUR CODE                               #
    ################################################################################
    return loss_op

The training loop in `cnn_rnn_training` should be identical to the training loop in `cnn_training`.

In [ ]:
def cnn_rnn_training(x_train, y_train, x_val, y_val, num_epochs, batch_size,
             save_model=True, model_dir='models/best_model'): 
    ################################################################################
    # Arguments:
    # - x_train: input training set
    # - y_train: label training set
    # - x_val: input validation set
    # - y_val: label validation set
    # - hyperparam_config: a dictionary that stores a hyperparameter configuration,
    #                      including:
    #                      - "dropout_rate": dropout rate (1 - keep probability),
    #                      - "l2": coefficient lambda for L2 regularization,
    #                      - "lr": learning rate for RMSProp optimizer
    # - num_epochs: number of epochs to train
    # - batch_size: training mini-batch size
    # - best_model_dir: location where model will be saved
    # Return values:
    # - best_loss: best validation loss
    ################################################################################
    tf.reset_default_graph()
    
    # define input and output placeholders
    # TF uses NHWC: the data is stored in the order of [batch, in_height, in_width, in_channels]
    # DNA is one-dimensional (height = 1, width = 101) with four channels for the four bases A, C, G, T
    x_ph = tf.placeholder("float", [None, 1, SEQ_LEN, NUM_BASES], name="x_ph")
    y_ph = tf.placeholder("float", [None, NUM_CLASSES], name="y_ph")

    # model
    y_hat_op, z_op = cnn_rnn_model(x_ph)
    
    # loss
    loss_op = cnn_rnn_loss(z_op, y_ph)

    # optimizer
    optimizer_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(loss_op)
     
    # Implement training loop
    # - loop through training data num_epochs times
    # - after each epoch, calculate loss on validation set (x_val, y_val)
    # - if save_model: save model with lowest validation loss to disk (model_dir)
    # - (delete previously best model)
    # - return lowest validation loss
    ################################################################################
    #                            BEGINNING OF YOUR CODE                            #
    ################################################################################
    
    
    
    ################################################################################
    #                               END OF YOUR CODE                               #
    ################################################################################
    return best_loss

### Performance evaluation

#### LSTM

In [ ]:
best_rnn_model_dir = "models/best_model_rnn"
validation_loss = cnn_rnn_training(x_tr_rnn, y_tr_rnn, x_va_rnn, y_va_rnn, num_epochs, batch_size, save_model = True,
                                   model_dir = best_rnn_model_dir)
print('validation loss:', validation_loss)

We can reuse the `predict` function from before.

In [ ]:
y_hat_test_rnn = predict(x_te_rnn, best_rnn_model_dir)

#### CNN

In [ ]:
best_cnn_model_dir = "models/best_model_cnn"
################################################################################
#                            BEGINNING OF YOUR CODE                            #
################################################################################
# change dropout rate, L2 coefficient, and learning rate to
# best values (according to grid search from previous problem):
hyperparam_config = {'dropout_rate': 0,
                     'l2': 0,
                     'lr': 0}
################################################################################
#                               END OF YOUR CODE                               #
################################################################################
validation_loss = training(x_tr_rnn, y_tr_rnn, x_va_rnn, y_va_rnn,
                           hyperparam_config, num_epochs, batch_size, save_model=True, model_dir=best_cnn_model_dir)
print('validation loss:', validation_loss)

In [ ]:
y_hat_test_cnn = predict(x_te_rnn, best_cnn_model_dir)

#### Model comparison

In [ ]:
def plot_roc_curve(y, y_hat, labels):
    # RNN
    fpr0, tpr0, _ = sklearn.metrics.roc_curve(y[0][:, 0], y_hat[0][:, 0])
    roc_auc0 = sklearn.metrics.auc(fpr0, tpr0)
    
    # CNN
    fpr1, tpr1, _ = sklearn.metrics.roc_curve(y[1][:, 0], y_hat[1][:, 0])
    roc_auc1 = sklearn.metrics.auc(fpr1, tpr1)

    plt.figure(figsize=(10,10))
    plt.rcParams.update({'font.size': 14})
    plt.plot(fpr0, tpr0, color='darkorange',
             lw=2, label=labels[0] + ' (area = %0.2f)' % roc_auc0)
    plt.plot(fpr1, tpr1, color='blue',
             lw=2, label=labels[1] + ' (area = %0.2f)' % roc_auc1)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

CNN auROC was around 0.85, and for LSTM around 0.87.

In [ ]:
plot_roc_curve([y_te_rnn, y_te_rnn], [y_hat_test_rnn, y_hat_test_cnn], ["CNN + LSTM", "CNN"])

In [ ]:
def plot_pr_curve(y, y_hat, labels):
    # RNN
    precision0, recall0, _ = sklearn.metrics.precision_recall_curve(y[0][:, 0], y_hat[0][:, 0])
    pr_auc0 = sklearn.metrics.auc(recall0, precision0)
    f10 = sklearn.metrics.f1_score(y[0][:, 0], np.round(y_hat[0][:, 0]))
    
    # CNN
    precision1, recall1, _ = sklearn.metrics.precision_recall_curve(y[1][:, 0], y_hat[1][:, 0])
    pr_auc1 = sklearn.metrics.auc(recall1, precision1)
    f11 = sklearn.metrics.f1_score(y[1][:, 0], np.round(y_hat[1][:, 0]))
    
    plt.figure(figsize=(10,10))
    plt.rcParams.update({'font.size': 14})
    plt.plot(recall0, precision0, color='darkorange',
             lw=2, label=labels[0] + ' (area = {0}, $F_1 = {1}$)'.format(round(pr_auc0, 2), round(f10, 2)))
    plt.plot(recall1, precision1, color='blue',
             lw=2, label=labels[1] + ' (area = {0}, $F_1 = {1}$)'.format(round(pr_auc1, 2), round(f11, 2)))

    f_scores = np.linspace(0.2, 0.8, num=4)
    lines = []
    labels = []
    for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
        plt.annotate('$F_1 = {0:0.1f}$'.format(f_score), xy=(0.87, y[45] + 0.02))

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.0])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve')
    plt.legend(loc="lower left")
    plt.show()

CNN auPR was around 0.79, and for LSTM around 0.81.

In [ ]:
plot_pr_curve([y_te_rnn, y_te_rnn], [y_hat_test_rnn, y_hat_test_cnn], ["CNN + LSTM", "CNN"])